# Content
* Importing Libraries
* Constants
* Preprocessing
    * Normalizing
    * Tokenizing
    * Stemming
    * Lemmatizing
* Feature Engineering
    * Bag of Words
    * FastText Word2Vec
* Model Selection

# Importing Libraries

In [2]:
from __future__ import unicode_literals

import json
import os
import numpy as np
import pandas as pd
from functools import reduce
from hazm import *
from pprint import pprint

# Constants

In [4]:
data_root = 'data'

keys = ['_id', 'NewsAgency', 'newsCode', 'newsLink', 'date',
        'newsPath', 'newsPathLinks', 'title', 'rutitr', 'subtitle',
  'body', 'bodyHtml', 'tags']

valid_tags = ['اجتماعی','اقتصادی','بین الملل','حوادث','خواندنی ها و دیدنی ها',
            'داستان کوتاه','روانشناسی','سرگرمی','سلامت','سیاست خارجی','سیاسی','علمی',
            'عمومی','فرهنگی/هنری','فناوری و IT','ورزشی']

news_agencies = ['AsrIran']

# Preprocessing

In [5]:
with open(os.path.join(data_root, 'out.jsonl'), encoding='utf-8') as json_data:
    news = [json.loads(line) for line in json_data]
    news = pd.DataFrame(news)
print('Number of Datapoints: {}'.format(len(news)))

Number of Datapoints: 1000


Lets look at our data

In [9]:
news.head(3)

,NewsAgency,_id,body,bodyHtml,date,newsCode,newsLink,newsPath,newsPathLinks,rutitr,subtitle,tags,title
0,AsrIran,5b4f7279020eb20597f401b4,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,"<img align=""left"" class=""news_corner_image"" s...",تاریخ انتشار: ۲۱:۲۲ - ۲۷ تير ۱۳۹۷ - 18 July 2018,621662,http://www.asriran.com/fa/news/621662,صفحه نخست » ورزشی,"{'صفحه نخست': '/fa/archive?service_id=1', 'ورز...",,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,5b4f7279020eb20597f401b5,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,<p><br/>دادستان انتظامی مالیاتی سازمان امور م...,تاریخ انتشار: ۲۱:۱۱ - ۲۷ تير ۱۳۹۷ - 18 July 2018,621659,http://www.asriran.com/fa/news/621659,صفحه نخست » اجتماعی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اجت...",,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...
2,AsrIran,5b4f727a020eb20597f401b6,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,<p>قیمت سبد نفتی اوپک دیروز به روند کاهشی خود...,تاریخ انتشار: ۲۱:۲۰ - ۲۷ تير ۱۳۹۷ - 18 July 2018,621661,http://www.asriran.com/fa/news/621661,صفحه نخست » اقتصادی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اقت...",,,"{'اوپک': '/fa/tag/1/اوپک', 'نفت': '/fa/tag/1/ن...",قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...


In [21]:
print(news.NewsAgency.unique()) # so we just add 'AsrIran' as news agency constants
print(news.subtitle.sample(5))
print(news.rutitr.sample())

['AsrIran']
761    زائران حج تمتع ۹۷ می‌توانند از امروز به دریافت...
999    یکی از شگردهای مدیران موسسات مالی غیرمجاز و با...
365    بر اساس گزارشی که این محققان امنیتی ارائه داده...
833                                                     
65     در این طرح کارکنان یکی از خطوط متروی وین اقدام...
Name: subtitle, dtype: object
228    تبعات تصمیم‌گیری یک‌جانیه مدیران آبی شانس جام ...
Name: rutitr, dtype: object


As we can see, there might not exist any 'subtitle' or 'rutitr', so we drop them if they do not have valuable features.

In [23]:
print("Not null 'subtitle' ",len([i for i in news.subtitle if len(i) != 0]))
print("Not null 'rutitr' ",len([i for i in news.rutitr if len(i) != 0]))

Not null 'subtitle'  569
Not null 'rutitr'  50


So based on information we got here, we know that these columns can help us, so we consider them.

But it is clear for us, `date`,`newsCode`, `newsLink`,`bodyHtml` and `_id` are useless features. So we remove them from our dataset.


In [26]:
news = news.drop(['_id','date','newsCode','newsLink','bodyHtml'], axis=1)

In [27]:
news.head(3)

,NewsAgency,body,newsPath,newsPathLinks,rutitr,subtitle,tags,title
0,AsrIran,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,صفحه نخست » ورزشی,"{'صفحه نخست': '/fa/archive?service_id=1', 'ورز...",,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,صفحه نخست » اجتماعی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اجت...",,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...
2,AsrIran,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,صفحه نخست » اقتصادی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اقت...",,,"{'اوپک': '/fa/tag/1/اوپک', 'نفت': '/fa/tag/1/ن...",قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...


In [73]:
newspathlinks_tags = list(set([list(x.keys())[1] for x in news.newsPathLinks]))
newspathlinks_tags

['سیاسی',
 'سلامت',
 'علمی',
 'اجتماعی',
 'بین الملل',
 'داستان کوتاه',
 'ورزشی',
 'روانشناسی',
 'اقتصادی',
 'سیاست خارجی',
 'عمومی',
 'فناوری و IT',
 'سرگرمی',
 'حوادث',
 'دانلود',
 'خواندنی ها و دیدنی ها',
 'فرهنگی/هنری']

In [83]:
newspath_tags = list(set(x.split(' » ')[1] for x in news.newsPath))
newspath_tags

['سیاسی',
 'سلامت',
 'علمی',
 'اجتماعی',
 'بین الملل',
 'داستان کوتاه',
 'ورزشی',
 'روانشناسی',
 'اقتصادی',
 'سیاست خارجی',
 'عمومی',
 'فناوری و IT',
 'سرگرمی',
 'حوادث',
 'دانلود',
 'خواندنی ها و دیدنی ها',
 'فرهنگی/هنری']

In [103]:
news.tags[85]

{'خشونت': '/fa/tag/1/خشونت',
 'زنان': '/fa/tag/1/زنان',
 'منع خشونت': '/fa/tag/1/منع خشونت'}

## Normalizing

In [38]:
normalizer = Normalizer()
news['body'] = news['body'].apply(normalizer.normalize)
news['rutitr'] = news['rutitr'].apply(normalizer.normalize)
news['subtitle'] = news['subtitle'].apply(normalizer.normalize)
news['title'] = news['title'].apply(normalizer.normalize)

## Tokenizing

In [40]:
def tokenize(phrase):
    sentences = sent_tokenize(phrase)
    if len(sentences) > 1:
        words = reduce(np.append, [word_tokenize(sentence) for sentence in sentences])
    elif len(sentences) == 1:
        words = word_tokenize(sentences[0])
    else:
        words = None
    return words

In [41]:
news['body'] = news['body'].apply(tokenize)
news['rutitr'] = news['rutitr'].apply(tokenize)
news['subtitle'] = news['subtitle'].apply(tokenize)
news['title'] = news['title'].apply(tokenize)

## Stemming

In [42]:
stemmer = Stemmer()
stem = lambda s: [stemmer.stem(w) for w in s] if s is not None else None
news['body'] = news['body'].apply(stem)
news['rutitr'] = news['rutitr'].apply(stem)
news['subtitle'] = news['subtitle'].apply(stem)
news['title'] = news['title'].apply(stem)

## Lemmatizing

In [43]:
lemmatizer = Lemmatizer()
lemmatize = lambda s: [lemmatizer.lemmatize(w) for w in s] if s is not None else None
news['body'] = news['body'].apply(lemmatize)
news['rutitr'] = news['rutitr'].apply(lemmatize)
news['subtitle'] = news['subtitle'].apply(lemmatize)
news['title'] = news['title'].apply(lemmatize)

# Feature Engineering

# Model Selection

## Naive Bayes

## Logistic Regression

## SVM

## Simple ANN

## CNN

## RNN

## CNN + RNN